<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=40ece1a6b9afe648d771497f893e3f0f95a838c3b23cea6d2a9736f3b43c2bef
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 10:11:43
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.45 C
-------------------
Today PnL: -70.24 K (-0.48%)
Current PnL: -18.76 L (-12.29%)
CY Booked + Current PnL: -6.63 L (-4.34%)
-------------------
Total profit:  2.59 L
Total loss:  -21.34 L
-------------------
Total Booked + Current PnL: 20.35 L (16.2%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.83 L (59.01%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.91,71.0,X-LC,4.57,233017.0,22651.0,6128.0,-0.70,10.77,2.63,13.68,37.0,3.70,1.62,29.07,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,106.40,58.0,M-SC,1.27,88949.0,-11828.0,11919.0,-2.51,-11.74,13.40,0.09,245.0,-0.99,0.62,16.86,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-16.77,73.0,H-MC,16.98,215551.0,35743.0,13601.0,-0.44,19.88,6.31,27.45,88.0,2.63,1.50,60.74,XR,NTT,BANKS
42,ITC,452.0,-36.23,64.0,X-LC,1.27,205502.0,5364.0,15536.0,-0.32,2.68,7.56,10.44,4.0,0.35,1.43,8.39,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-10.05,39.0,X-MC,2.51,88830.0,-1848.0,16753.0,-0.32,-2.04,18.86,16.44,101.0,-0.11,0.62,17.73,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,67.49,70.0,H-SC,3.75,145309.0,-37466.0,148535.0,7.39,-20.50,102.22,60.77,125.0,-0.25,1.01,30.15,XR,NTT,JEWELLERY
17,CAMS,5211.76,-2.95,56.0,X-SC,0.52,110337.0,8333.0,35595.0,2.21,8.17,32.26,43.06,122.0,0.23,0.77,26.68,X40N,ATH,MISC
50,MASFIN,398.61,-18.25,54.0,H-SC,9.18,92700.0,-5280.0,26883.0,1.85,-5.39,29.00,22.05,152.0,-0.20,0.64,34.75,XR,ATH,FINANCE
83,VALIANTORG,838.00,-291.52,46.0,H-SC,9.11,106335.0,-57270.0,178590.0,1.18,-35.00,167.95,74.15,139.0,-0.32,0.74,36.30,XR,NTT,CHEMICALS
19,COALINDIA,497.97,7.46,46.0,L-LC,2.41,142339.0,-8122.0,41905.0,0.71,-5.40,29.44,22.46,167.0,-0.19,0.99,11.70,XY25,ATH,MINING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,106.40,58.0,M-SC,1.27,88949.0,-11828.0,11919.0,-2.51,-11.74,13.40,0.09,245.0,-0.99,0.62,16.86,OX40N,NTT,DURABLES
84,VBL,671.64,-13.87,62.0,X-LC,8.47,310132.0,-5710.0,118377.0,-1.89,-1.81,38.17,35.67,5.0,-0.05,2.15,11.80,X40N,ATH,FMCG
61,ROUTE,2236.84,-55.56,30.0,H-SC,22.17,73023.0,-60043.0,152896.0,-1.87,-45.12,209.38,69.78,118.0,-0.39,0.51,0.02,SR,ATH,IT
62,SAIL,228.00,50.76,61.0,M-MC,10.27,237498.0,12536.0,155110.0,-1.87,5.57,65.31,74.53,192.0,0.08,1.65,39.62,XY24,BTT,STEEL
36,IEX,219.00,-30.39,58.0,H-SC,15.94,198597.0,-1357.0,98584.0,-1.55,-0.68,49.64,48.63,115.0,-0.01,1.38,10.80,XR,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-30.39,58.0,H-SC,15.94,198597.0,-1357.0,98584.0,-1.55,-0.68,49.64,48.63,115.0,-0.01,1.38,10.80,XR,NTT,MISC
87,WIPRO,420.0,-14.70,44.0,M-LC,5.69,149618.0,-1327.0,110777.0,-0.40,-0.88,74.04,72.51,53.0,-0.01,1.04,5.09,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.40,58.0,M-SC,1.27,88949.0,-11828.0,11919.0,-2.51,-11.74,13.40,0.09,245.0,-0.99,0.62,16.86,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,51.0,H-SC,2.82,225108.0,-44559.0,80881.0,-1.01,-16.52,35.93,13.47,138.0,-0.55,1.56,14.46,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.37,54.0,H-MC,6.43,105070.0,-26765.0,68989.0,-0.15,-20.30,65.66,32.03,98.0,-0.39,0.73,17.69,OX40N,NTT,IT
18,CERA,9475.0,-23.15,48.0,H-SC,2.51,140001.0,-35902.0,77925.0,-0.64,-20.41,55.66,23.89,149.0,-0.46,0.97,20.91,OX40N,NTT,CERAMICS
68,SIS,528.0,2024.35,44.0,H-SC,3.21,85731.0,-25301.0,48910.0,-1.13,-22.79,57.05,21.26,156.0,-0.52,0.60,15.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,50.76,61.0,M-MC,10.27,237498.0,12536.0,155110.0,-1.87,5.57,65.31,74.53,192.0,0.08,1.65,39.62,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,58.0,M-LC,5.17,209522.0,6352.0,35786.0,-1.19,3.13,17.08,20.74,55.0,0.18,1.46,10.20,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.88,49.0,M-LC,11.33,208126.0,3050.0,109516.0,-1.06,1.49,52.62,54.89,52.0,0.03,1.45,25.94,X5K,ATH,METALS
47,KPIGREEN,731.05,11.71,71.0,H-SC,5.46,132930.0,7633.0,51298.0,-0.79,6.09,38.59,47.03,141.0,0.15,0.92,66.44,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,61.0,M-LC,9.67,165036.0,24196.0,20299.0,-0.66,17.18,12.30,31.59,66.0,1.19,1.15,48.22,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-16.77,73.0,H-MC,16.98,215551.0,35743.0,13601.0,-0.44,19.88,6.31,27.45,88.0,2.63,1.50,60.74,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.71,71.0,H-SC,5.46,132930.0,7633.0,51298.0,-0.79,6.09,38.59,47.03,141.0,0.15,0.92,66.44,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,55.0,H-SC,6.66,130062.0,6726.0,124899.0,-1.20,5.45,96.03,106.72,119.0,0.05,0.90,30.11,AR,ATH,MISC
32,HINDZINC,730.22,27.88,49.0,M-LC,11.33,208126.0,3050.0,109516.0,-1.06,1.49,52.62,54.89,52.0,0.03,1.45,25.94,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.04,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.93,89466.0,-39204.0,84501.0,0.08,-30.47,94.45,35.20,219.0,-0.46,0.62,2.69,X40,NTT,FOOTWEAR
24,DMART,5391.45,-17.39,35.0,X-LC,9.66,101016.0,-1570.0,28375.0,-0.48,-1.53,28.09,26.13,38.0,-0.06,0.70,23.65,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.48,37.0,X-MC,5.24,199395.0,-952.0,31465.0,-0.27,-0.48,15.78,15.23,178.0,-0.03,1.38,31.62,X40N,NTT,REALTY
53,PGHH,17907.65,-30.81,37.0,X-MC,2.28,200430.0,-390.0,68186.0,-0.68,-0.19,34.02,33.76,80.0,-0.01,1.39,4.99,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.18,49.0,H-LC,1.17,157075.0,-29020.0,76496.0,-0.23,-15.59,48.70,25.51,15.0,-0.38,1.09,15.75,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.23,64.0,X-LC,1.27,205502.0,5364.0,15536.0,-0.32,2.68,7.56,10.44,4.0,0.35,1.43,8.39,X40,NTT,FMCG
53,PGHH,17907.65,-30.81,37.0,X-MC,2.28,200430.0,-390.0,68186.0,-0.68,-0.19,34.02,33.76,80.0,-0.01,1.39,4.99,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.33,50.0,X-MC,3.00,315840.0,-12151.0,118693.0,-0.36,-3.70,37.58,32.48,92.0,-0.10,2.19,7.30,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,0.32,49.0,X-MC,4.17,210600.0,18858.0,18891.0,-1.04,9.84,8.97,19.69,99.0,1.00,1.46,17.30,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.93,89466.0,-39204.0,84501.0,0.08,-30.47,94.45,35.20,219.0,-0.46,0.62,2.69,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.78,40.0,X-MC,6.69,81570.0,-1940.0,21967.0,-1.39,-2.32,26.93,23.98,82.0,-0.09,0.57,3.36,X40,ATH,APPARELS
21,COLPAL,3767.14,-3.88,48.0,X-MC,7.68,224450.0,-38915.0,152267.0,-1.02,-14.78,67.84,43.04,84.0,-0.26,1.56,4.20,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-13.63,51.0,X-MC,5.87,181892.0,-18614.0,115774.0,0.23,-9.28,63.65,48.45,90.0,-0.16,1.26,4.89,X40N,ATH,FINANCE
53,PGHH,17907.65,-30.81,37.0,X-MC,2.28,200430.0,-390.0,68186.0,-0.68,-0.19,34.02,33.76,80.0,-0.01,1.39,4.99,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.90,51.0,X-SC,37.93,54454.0,-10552.0,161478.0,-1.06,-16.23,296.54,232.18,198.0,-0.07,0.38,6.22,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.78,40.0,X-MC,6.69,81570.0,-1940.0,21967.0,-1.39,-2.32,26.93,23.98,82.0,-0.09,0.57,3.36,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.59,45.0,X-SC,4.77,82325.0,-62835.0,138759.0,-0.66,-43.29,168.55,52.31,136.0,-0.45,0.57,10.20,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,39.0,X-MC,2.51,88830.0,-1848.0,16753.0,-0.32,-2.04,18.86,16.44,101.0,-0.11,0.62,17.73,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.93,89466.0,-39204.0,84501.0,0.08,-30.47,94.45,35.20,219.0,-0.46,0.62,2.69,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.04,53.0,X-LC,10.96,286230.0,-59726.0,126428.0,-0.41,-17.26,44.17,19.28,1.0,-0.47,1.99,5.81,X40,ATH,IT
40,INFY,2275.00,-18.52,55.0,X-LC,5.80,317839.0,4951.0,166738.0,-1.20,1.58,52.46,54.87,3.0,0.03,2.21,10.00,X40,BTT,IT
42,ITC,452.00,-36.23,64.0,X-LC,1.27,205502.0,5364.0,15536.0,-0.32,2.68,7.56,10.44,4.0,0.35,1.43,8.39,X40,NTT,FMCG
84,VBL,671.64,-13.87,62.0,X-LC,8.47,310132.0,-5710.0,118377.0,-1.89,-1.81,38.17,35.67,5.0,-0.05,2.15,11.80,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,55.0,H-LC,5.61,252840.0,-8779.0,127988.0,-0.41,-3.36,50.62,45.57,7.0,-0.07,1.76,7.59,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7022.22,59.0,L-SC,7.85,78173.0,-15637.0,92252.0,-0.38,-16.67,118.01,81.67,269.0,-0.17,0.54,51.39,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,47.0,L-SC,28.15,80582.0,-32967.0,73015.0,-0.78,-29.03,90.61,35.27,268.0,-0.45,0.56,97.23,XR,NTT,HOTELS
50,MASFIN,398.61,-18.25,54.0,H-SC,9.18,92700.0,-5280.0,26883.0,1.85,-5.39,29.00,22.05,152.0,-0.20,0.64,34.75,XR,ATH,FINANCE
37,INDIAMART,4810.62,-53.33,55.0,H-SC,6.66,130062.0,6726.0,124899.0,-1.20,5.45,96.03,106.72,119.0,0.05,0.90,30.11,AR,ATH,MISC
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,67.49,70.0,H-SC,3.75,145309.0,-37466.0,148535.0,7.39,-20.50,102.22,60.77,125.0,-0.25,1.01,30.15,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,11.71,71.0,H-SC,5.46,132930.0,7633.0,51298.0,-0.79,6.09,38.59,47.03,141.0,0.15,0.92,66.44,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES
69,SONACOMS,806.63,-31.93,71.0,M-SC,7.81,84656.0,-16503.0,56508.0,0.12,-16.31,66.75,39.54,202.0,-0.29,0.59,19.37,AR,ATH,AUTO
5,ANGELONE,3033.00,15.82,63.0,X-SC,5.56,205287.0,14281.0,43418.0,-0.33,7.48,21.15,30.21,157.0,0.33,1.43,29.92,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.00
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.08
LC    32.36
MC    24.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.05
X40      16.21
XY25     12.03
XR       11.45
X40N     10.69
AR        7.84
OX40N     6.53
X200      1.77
X5K       1.45
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.45
X-LC    21.65
X-MC    18.72
M-SC    12.65
M-LC     5.10
H-LC     4.62
X-SC     4.55
H-MC     3.88
M-MC     1.65
L-SC     1.43
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.52,-3.06,36.69
IT,12.47,-17.19,80.05
FINANCE,12.17,-7.88,56.79
MISC,6.85,-16.69,81.34
ELECTRICAL,5.82,-7.86,46.10
BANKS,5.66,-5.03,59.35
PAINTS,5.49,-16.84,34.25
INSURANCE,3.76,-2.09,40.16
AUTO,2.77,-44.47,104.76


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246457.0,22
XR,1236539.0,14
AR,1160630.0,9
X40,823779.0,12
X40N,620610.0,9
OX40N,544774.0,9
XY25,451788.0,8
SR,267281.0,2
X5K,109516.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3310279.0,24
M-SC,1223981.0,15
X-MC,1181201.0,14
X-LC,1022245.0,13
X-SC,526793.0,6
M-LC,276378.0,4
H-LC,274365.0,3
H-MC,258620.0,3
L-SC,254677.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1152478.0      6
           AR         833256.0      5
           XR         763643.0      7
M-SC       XY24       702551.0      6
X-MC       XY24       575489.0      4
X-LC       X40        451963.0      5
           XY24       303022.0      2
X-MC       X40        287315.0      6
X-SC       X40N       280814.0      4
H-SC       SR         267281.0      2
           OX40N      242323.0      3
H-LC       AR         204484.0      2
X-LC       X40N       192557.0      3
H-MC       XY24       176030.0      1
X-MC       XY25       171158.0      2
L-SC       XR         165267.0      2
X-SC       XY24       161478.0      1
M-MC       XY24       155110.0      1
X-MC       X40N       147239.0      2
M-SC       OX40N      144052.0      4
           XY25       128236.0      1
           XR         126252.0      2
           AR         122890.0      2
M-LC       XR         110777.0      1
           X5K        109516.0      1
L-SC       OX40N       89410.0      1
X-SC       X40         84501.0      1
X-LC       XY25        74703.0      3
H-LC       X200        69881.0      1
H-MC       OX40N       68989.0      1
L-MC       XR          56999.0      1
H-SC       MH          51298.0      1
L-LC       XY25        41905.0      1
M-LC       XY25        35786.0      1
           XY24        20299.0      1
H-MC       XR          13601.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
